In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.3 MB/s eta 0:00:00


In [25]:
pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.4 MB/s eta 0:00:00


In [32]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model, Sequential, load_model
from transformers import TFBertModel
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
df = pd.read_csv('/content/drive/MyDrive/HATE_SPEECH_DETECTION_PROJECT/DATASET/unlabelled_data_tweets.csv')

In [4]:
df

,Unnamed: 0,tweets
0,0,RT @TheLynch10: This grifting bastard can pay ...
1,1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTEK...
2,2,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTEK...
3,3,RT @johnnyd_ee: All these idiots claiming that...
4,4,RT @johnnyd_ee: All these idiots claiming that...
...,...,...
1995,1995,😂 https://t.co/v6OhRZRoq8
1996,1996,Mark really has fun at home 😂 https://t.co/Ssb...
1997,1997,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey's...
1998,1998,祝福全天下的父親..... https://t.co/b7NRE5AgmV


In [5]:
df.drop_duplicates(subset=['tweets'], inplace=True)

In [6]:
df.shape

(1285, 2)

In [7]:
df['tweets'] = df['tweets'].str.replace('@', ' @')
df['tweets'] = df['tweets'].str.replace('#', ' #')

In [8]:
#Code to remove @
df['clean_tweets'] = df['tweets'].apply(lambda x : ' '.join([tweets for tweets in x.split()if not tweets.startswith("@")]))

In [9]:
# Function to remove URLs from a string

url_pattern = re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')# Define URL regex pattern

def remove_urls(text):
    return url_pattern.sub('', text)

# Apply function to 'text' column
df['clean_tweets'] = df['clean_tweets'].apply(remove_urls)

In [10]:
df

,Unnamed: 0,tweets,clean_tweets
0,0,RT @TheLynch10: This grifting bastard can pay...,RT This grifting bastard can pay his own fucki...
1,1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTE...,RT KAMPANYAMIZ SİZLERİN DESTEKLERİ SAYESİNDE %...
3,3,RT @johnnyd_ee: All these idiots claiming tha...,RT All these idiots claiming that #BorisJohnso...
5,5,"RT @911Amelia: Let be clear, PM Sunak has ref...","RT Let be clear, PM Sunak has referred #BorisJ..."
6,6,RT @911Amelia: #BorisJohnson distanced himse...,RT #BorisJohnson distanced himself from the ca...
...,...,...,...
1995,1995,😂 https://t.co/v6OhRZRoq8,😂
1996,1996,Mark really has fun at home 😂 https://t.co/Ssb...,Mark really has fun at home 😂
1997,1997,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey'...,RT TUANZY HIGHLIGHT! from joey's twitch livest...
1998,1998,祝福全天下的父親..... https://t.co/b7NRE5AgmV,祝福全天下的父親.....


In [11]:
#Removing numbers
df['clean_tweets'] = df['clean_tweets'].apply(lambda x : ' '.join([Text for Text in x.split() if not Text == '\d*']))
df['clean_tweets'] = df['clean_tweets'].apply(lambda x: ''.join(char for char in x if not char.isnumeric()))

In [12]:
df['clean_tweets'] = df['clean_tweets'].apply(lambda x: ''.join(char for char in x if char.isalnum() or char.isspace()))

In [13]:
df

,Unnamed: 0,tweets,clean_tweets
0,0,RT @TheLynch10: This grifting bastard can pay...,RT This grifting bastard can pay his own fucki...
1,1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTE...,RT KAMPANYAMIZ SİZLERİN DESTEKLERİ SAYESİNDE ...
3,3,RT @johnnyd_ee: All these idiots claiming tha...,RT All these idiots claiming that BorisJohnson...
5,5,"RT @911Amelia: Let be clear, PM Sunak has ref...",RT Let be clear PM Sunak has referred BorisJoh...
6,6,RT @911Amelia: #BorisJohnson distanced himse...,RT BorisJohnson distanced himself from the cab...
...,...,...,...
1995,1995,😂 https://t.co/v6OhRZRoq8,
1996,1996,Mark really has fun at home 😂 https://t.co/Ssb...,Mark really has fun at home
1997,1997,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey'...,RT TUANZY HIGHLIGHT from joeys twitch livestre...
1998,1998,祝福全天下的父親..... https://t.co/b7NRE5AgmV,祝福全天下的父親


In [14]:
def remove_non_english(text):
    return re.sub(r'[^\x00-\x7F]', '', text)

df['clean_tweets'] = df['clean_tweets'].apply(remove_non_english)

In [15]:
df

,Unnamed: 0,tweets,clean_tweets
0,0,RT @TheLynch10: This grifting bastard can pay...,RT This grifting bastard can pay his own fucki...
1,1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTE...,RT KAMPANYAMIZ SZLERN DESTEKLER SAYESNDE OLDU...
3,3,RT @johnnyd_ee: All these idiots claiming tha...,RT All these idiots claiming that BorisJohnson...
5,5,"RT @911Amelia: Let be clear, PM Sunak has ref...",RT Let be clear PM Sunak has referred BorisJoh...
6,6,RT @911Amelia: #BorisJohnson distanced himse...,RT BorisJohnson distanced himself from the cab...
...,...,...,...
1995,1995,😂 https://t.co/v6OhRZRoq8,
1996,1996,Mark really has fun at home 😂 https://t.co/Ssb...,Mark really has fun at home
1997,1997,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey'...,RT TUANZY HIGHLIGHT from joeys twitch livestre...
1998,1998,祝福全天下的父親..... https://t.co/b7NRE5AgmV,


In [16]:
def remove_words(text, words_to_remove):
    for word in words_to_remove:
        text = text.replace(word, '')
    return text

# List of words or characters to remove
words_to_remove = ['RT']

# Apply the function to the 'Text' column
df['clean_tweets'] = df['clean_tweets'].apply(lambda x: remove_words(x, words_to_remove))

In [17]:
df

,Unnamed: 0,tweets,clean_tweets
0,0,RT @TheLynch10: This grifting bastard can pay...,This grifting bastard can pay his own fucking...
1,1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTE...,KAMPANYAMIZ SZLERN DESTEKLER SAYESNDE OLDU B...
3,3,RT @johnnyd_ee: All these idiots claiming tha...,All these idiots claiming that BorisJohnson a...
5,5,"RT @911Amelia: Let be clear, PM Sunak has ref...",Let be clear PM Sunak has referred BorisJohns...
6,6,RT @911Amelia: #BorisJohnson distanced himse...,BorisJohnson distanced himself from the cabin...
...,...,...,...
1995,1995,😂 https://t.co/v6OhRZRoq8,
1996,1996,Mark really has fun at home 😂 https://t.co/Ssb...,Mark really has fun at home
1997,1997,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey'...,TUANZY HIGHLIGHT from joeys twitch livestream...
1998,1998,祝福全天下的父親..... https://t.co/b7NRE5AgmV,


In [18]:
df = df.drop('Unnamed: 0', axis=1)

In [19]:
df = df.reset_index(drop=True)

In [20]:
df

,tweets,clean_tweets
0,RT @TheLynch10: This grifting bastard can pay...,This grifting bastard can pay his own fucking...
1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTE...,KAMPANYAMIZ SZLERN DESTEKLER SAYESNDE OLDU B...
2,RT @johnnyd_ee: All these idiots claiming tha...,All these idiots claiming that BorisJohnson a...
3,"RT @911Amelia: Let be clear, PM Sunak has ref...",Let be clear PM Sunak has referred BorisJohns...
4,RT @911Amelia: #BorisJohnson distanced himse...,BorisJohnson distanced himself from the cabin...
...,...,...
1280,😂 https://t.co/v6OhRZRoq8,
1281,Mark really has fun at home 😂 https://t.co/Ssb...,Mark really has fun at home
1282,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey'...,TUANZY HIGHLIGHT from joeys twitch livestream...
1283,祝福全天下的父親..... https://t.co/b7NRE5AgmV,


In [21]:
df = df.dropna(subset=['clean_tweets'])
df = df[df['clean_tweets'].str.strip() != '']

In [22]:
df = df.reset_index(drop=True)

In [30]:
df

,tweets,clean_tweets
0,RT @TheLynch10: This grifting bastard can pay...,This grifting bastard can pay his own fucking...
1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTE...,KAMPANYAMIZ SZLERN DESTEKLER SAYESNDE OLDU B...
2,RT @johnnyd_ee: All these idiots claiming tha...,All these idiots claiming that BorisJohnson a...
3,"RT @911Amelia: Let be clear, PM Sunak has ref...",Let be clear PM Sunak has referred BorisJohns...
4,RT @911Amelia: #BorisJohnson distanced himse...,BorisJohnson distanced himself from the cabin...
...,...,...
1112,"@newsenstar @locha1118 Time too short , he w...",Time too short he went back
1113,"RT @newsenstar: [V LIVE] 갓세븐 마크, 꼼꼼한 입국 심사에 시...",V LIVE TV mark GOT Arrival Inch
1114,Mark really has fun at home 😂 https://t.co/Ssb...,Mark really has fun at home
1115,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey'...,TUANZY HIGHLIGHT from joeys twitch livestream...


IMPORTING THE MODEL

In [27]:
custom_objects = {'KerasLayer': hub.KerasLayer, 'TFBertModel': TFBertModel}
model=load_model( "/content/drive/MyDrive/HATE_SPEECH_DETECTION_PROJECT/models/TensorFlow_BERT_model_v-1:4.h5", custom_objects=custom_objects)

In [31]:
hate_percent = model.predict([df['clean_tweets']])
print(hate_percent)

35/35 [==============================] - 10s 159ms/step
[[0.719345  ]
 [0.1530626 ]
 [0.6009545 ]
 ...
 [0.07660332]
 [0.4374679 ]
 [0.05635297]]


In [33]:
df['label'] = np.where(hate_percent > 0.5, 1, 0)

In [35]:
df

,tweets,clean_tweets,label
0,RT @TheLynch10: This grifting bastard can pay...,This grifting bastard can pay his own fucking...,1
1,RT @smayusufbaran: KAMPANYAMIZ SİZLERİN DESTE...,KAMPANYAMIZ SZLERN DESTEKLER SAYESNDE OLDU B...,0
2,RT @johnnyd_ee: All these idiots claiming tha...,All these idiots claiming that BorisJohnson a...,1
3,"RT @911Amelia: Let be clear, PM Sunak has ref...",Let be clear PM Sunak has referred BorisJohns...,0
4,RT @911Amelia: #BorisJohnson distanced himse...,BorisJohnson distanced himself from the cabin...,0
...,...,...,...
1112,"@newsenstar @locha1118 Time too short , he w...",Time too short he went back,0
1113,"RT @newsenstar: [V LIVE] 갓세븐 마크, 꼼꼼한 입국 심사에 시...",V LIVE TV mark GOT Arrival Inch,0
1114,Mark really has fun at home 😂 https://t.co/Ssb...,Mark really has fun at home,0
1115,RT @ONPOINT_MP: TUANZY HIGHLIGHT!\nfrom joey'...,TUANZY HIGHLIGHT from joeys twitch livestream...,0


In [36]:
df[df['label'] == 1]

,tweets,clean_tweets,label
0,RT @TheLynch10: This grifting bastard can pay...,This grifting bastard can pay his own fucking...,1
2,RT @johnnyd_ee: All these idiots claiming tha...,All these idiots claiming that BorisJohnson a...,1
5,RT @911Amelia: The Civil Service must have tr...,The Civil Service must have trust amp confide...,1
20,RT @CourierBoyUK: Haha! Rachel Johnson doesn’...,Haha Rachel Johnson doesnt work for the Gover...,1
24,RT @CharmaineScamm1: Government’s fight to wi...,Governments fight to withhold evidence from C...,1
...,...,...,...
1063,@marktuan I hope 2021 will not be the same as...,I hope will not be the same as last year n Le...,1
1083,No matter there is good weather/good course n ...,No matter there is good weathergood course n t...,1
1085,@byeYukio7 @abigatorrr_ @_MarksJourney @jo...,No problems Im leftover lover,1
1099,@mtuan93 Chicken eats chicken...Cannibalism 🤣🤣,Chicken eats chickenCannibalism,1


In [38]:
df.to_csv('/content/drive/MyDrive/HATE_SPEECH_DETECTION_PROJECT/DATASET/TESTED_labelled_data_tweets.csv')